<a href="https://colab.research.google.com/github/nickxr/CudaLabs/blob/main/Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Thu Sep  9 18:19:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
torch.cuda.is_available()
# Output would be True if Pytorch is using GPU otherwise it would be False.

True

Полностью удалим все предыдущие версии CUDA. Нам нужно обновить облачный экземпляр CUDA.

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Установим CUDA версии 9 

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

Проверим, какая версия CUDA у нас установлена

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


Установим расширение CUDA для jupyter

In [2]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-nw3vputp
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-nw3vputp
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=9d2781e3688de247e25ee46f7f68890229ae813f452b9b032432e09c5e76dc69
  Stored in directory: /tmp/pip-ephem-wheel-cache-5_foxc3y/wheels/c5/2b/c0/87008e795a14bbcdfc7c846a00d06981916331eb980b6c8bdf
Successfully built NVCCPlugin


Загрузим расширение

In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


Напишем Hello World, чтобы проверить, что плагин работает

In [5]:
%%cu
#include <iostream>
    int
    main()
{
    std::cout << "Hello World\n";
    return 0;
}

Hello World



Задание 7. Вычислить функцию синус икс делить на икс.

In [ ]:
%%cu
//подключение библиотек
#include <cuda_runtime.h>
#include "device_launch_parameters.h"
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

#define N (1024*1024)

//ядро
__global__ void sinus(float *x, float *res)
{
    *res = (float)__sinf(*x)/((float)*x);
}

//главная функция  
int main()
{
  //переменные на CPU
  float x, res;

  //переменные на GPU
  float *dev_x, *dev_res; 

  //размерность
  int size = sizeof(float);

  // выделяем память на GPU
  cudaMalloc( (void**)&dev_x, size );
  cudaMalloc( (void**)&dev_res, size );

  //инициализация переменных
  x = 0.5;
  res = 0.0;
 

  // копирование информации с CPU на GPU
  cudaMemcpy(dev_x, &x, size, cudaMemcpyHostToDevice);
  cudaMemcpy(dev_res, &res, size, cudaMemcpyHostToDevice);

  // вызов ядра
  sinus<<<size/256, 256>>>( dev_x, dev_res );

  //копирование результата работы ядра с GPU на CPU
  cudaMemcpy( &res, dev_res, size, cudaMemcpyDeviceToHost);

  //вывод информации
  printf("%f",res);

  // очищение памяти на GPU
  cudaFree( dev_x );
  cudaFree( dev_res );

  return 0;
}